<a href="https://colab.research.google.com/github/snehakokil/JHU-Notebooks/blob/main/Sentiment_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#!pip install pandas matplotlib scikit-learn numpy nltk
#!pip install --upgrade nltk
#!pip install transformers
#!pip install --upgrade datasets
#!pip install --upgrade huggingface_hub
! pip install PyTorch    
! pip install langchain
! pip install transformers
! pip install langchain-huggingface

import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np
import ast

# for NLP tasks
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import unicodedata
import re

# for prompt engineering


print("hello world")

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for PyTorch (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      Traceback (most recent call last):
        File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
          main()
        File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_proc

## Task 1: Load and Explore the Dataset
The given dataset is in a text file. I converted the text file into CSV for easier processing and loaded into a Pandas Dataframe. Next, I cleaned the dataset by filling in the empty values with a blank string. 

In [ ]:
# load the sentiment analysis dataset from a .txt file and save it as a .csv file
# Clean the data and handle potential parsing issues
# Path to your input .txt file
input_file = "Product_Sentiment.txt"
output_file = "Product_Sentiment.csv"

data = []

with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip().rstrip(",")  # remove trailing comma
        if line:  # skip empty lines
            try:
                # Safely evaluate the tuple string into a Python tuple
                text, label = ast.literal_eval(line)
                data.append((text, label))
            except Exception as e:
                print(f"Skipping line due to error: {line} -> {e}")

# Create DataFrame
df = pd.DataFrame(data, columns=["text", "label"])

# Save as CSV
df.to_csv(output_file, index=False, encoding="utf-8")

print("✅ Dataset loaded and saved as CSV!")
print(df.head())

# read the csv file and clean the data to remove any rows with missing or null values
df = pd.read_csv("Product_Sentiment.csv")
df.fillna('', inplace=True)
df.reset_index(drop=True, inplace=True) # reset index after dropping rows   
print("✅ Data cleaned!")
print(df.head())


## Task 2: Build a Traditional ML Classifier
For this we first need to split data into training and testing. From the existing dataset, I chose to reserve 80% of the data for training and 20% for testing. 

### Prepare training and testing datasets

In [ ]:
# Load your dataset (CSV with columns: "text", "label")
df = pd.read_csv("Product_Sentiment.csv")

# Split into train (80%) and test (20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Save to CSV files
train_df.to_csv("train.csv", index=False, encoding="utf-8")
test_df.to_csv("test.csv", index=False, encoding="utf-8")

print("✅ Dataset successfully split!")
print("Train size:", len(train_df))
print("Test size:", len(test_df))

### Create SVM classifier and train. Then apply the test data
(Used ChatGPT to understand steps)
This task requires building a pipeline, where the text data is converted into numerical features and then passed onto SVM classifier training and testing functions.

To convert text into numerical features, ChatGPT recommended using the TF-IDF Vectorizer. It also suggested the use of base classifier LinearSVC, which is known for its text classification capabilities.

# 

In [ ]:


# Load train and test datasets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Build a pipeline: TF-IDF vectorizer + Linear SVM
# svm_pipeline = Pipeline([
#    ('tfidf', TfidfVectorizer(stop_words='english')),
#    ('svm', LinearSVC(random_state=42))
# ])
# Load dataset
df = pd.read_csv("Product_Sentiment.csv")

# Define stratified k-fold (preserves class balance)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# CountVectorizer can also be used instead of TfidfVectorizer
svm_pipeline = Pipeline([
     ('count', CountVectorizer(stop_words='english')),
     ('svm', LinearSVC(random_state=42))
])

svm_scores = cross_val_score(svm_pipeline, df['text'], df['label'], cv=kfold, scoring='accuracy')
print("SVM + BoW - Accuracy per fold:", svm_scores)
print("SVM + BoW - Mean Accuracy:", svm_scores.mean())

# Train the model
svm_pipeline.fit(df['text'], df['label'])

# Evaluate on test set
#y_pred = svm_pipeline.predict(test_df['text'])
#print("✅ Accuracy:", accuracy_score(test_df['label'], y_pred))
#print("\nClassification Report:\n", classification_report(test_df['label'], y_pred))

# Example prediction
example = ["I really love this new phone!", "This is the worst product I’ve ever bought."]
predictions = svm_pipeline.predict(example)
for text, label in zip(example, predictions):
    print(f"Text: {text} -> Predicted Sentiment: {label}")

### Analysis of SVM with TF-IDF vectorizer
Looking at the results, especially the precision score of 0.00, the model performed poorly on picking up positive sentiments, while it did a bit better on negative ones. In spite of the performance scores, the predictions it generated on examples were accurate.

When analyzed with the help of ChatGPT, I understood that the TF-IDF vectorizer usually works better with larger datasets. This dataset being very small, became unstable and could not represent the positive sentiment test data very well.

ChatGPT also suggested using cross-validation to improve accuracy score on a small dataset, instead of 80-20 split. Another option was to try using simpler models like Naive Bayes.

I will be first trying to improve performance with Bag of Words approach and then test the suggestions given above.

### Analysis of SVM with Count Vectorizer
When replaced with this to implement Bag of Words approach, the accuracy score went up a little bit (50% from 37%), recall improved to 80%, because of smaller, imbalanced dataset - which means the model will rarely miss negative sentiments, sometimes even considering a positive sentiment as negative.

### Alterntive: Naive Bayes
Let's see how this model works. 

when ran this model with Bag of Words, it still gave out 37% accuracy, which was no improvement from the previous SVM implementation. In fact, compared to this, SVM with BOW gave better accuracy. (same as SVM + TF-IDF, and worse than SVM + BoW at 50%)

### Adding k-fold cross validation
Using this, I am trying to balance the train and test datasets in the hopes to get better precision and accuracy, if possible.

For this, I modified the SVM + BoW implementation above.

This approach improved the mean accuracy over 5 k-folds to *67%*


In [ ]:
# Load train and test datasets
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# CountVectorizer can also be used instead of TfidfVectorizer
nb_pipeline = Pipeline([
     ('count', CountVectorizer(stop_words='english')),
     ('nb', MultinomialNB())
])

# Train the model
nb_pipeline.fit(train_df['text'], train_df['label'])

# Evaluate on test set
y_pred = nb_pipeline.predict(test_df['text'])
print("✅ Accuracy:", accuracy_score(test_df['label'], y_pred))
print("\nClassification Report:\n", classification_report(test_df['label'], y_pred))

# Example prediction
example = ["I really love this new phone!", "This is the worst product I’ve ever bought."]
predictions = nb_pipeline.predict(example)
for text, label in zip(example, predictions):
    print(f"Text: {text} -> Predicted Sentiment: {label}")

## Task 3: Experiment with a Traditional NLP Approach
To use NLP techniques, I first need to pre-process the data to tokenize it, remove stopwords from it and then apply lemmatization. For that, the *nltk* library functions are used.

The following block contains functions supporting the data pre-processing done later.

In [ ]:
# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

# Initialising Stopwords and Lemmatizer

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def remove_non_ascii(words):
    """Remove non-ASCII characters from a list of tokenized words"""
    return [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in words]

def to_lowercase(words):
    """Convert all words to lowercase"""
    return [word.lower() for word in words]

def remove_punctuation(words):
    """Remove punctuation"""
    return [re.sub(r'[^\w\s]', '', word) for word in words if re.sub(r'[^\w\s]', '', word) != '']

def remove_stopwords(words):
    """Remove stopwords"""
    return [word for word in words if word not in stop_words]

def lemmatize_list(words):
    """Lemmatize words"""
    return [lemmatizer.lemmatize(word, pos='v') for word in words]

def normalize(text):
    """Full preprocessing pipeline"""
    words = nltk.word_tokenize(text)  # Tokenize text
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize_list(words)
    return ' '.join(words)

# Load your dataset (CSV with columns: "text", "label")
df = pd.read_csv("sample_data/Product_Sentiment.csv")

# Apply normalization to the text data
df['cleaned_text'] = df['text'].apply(normalize)
df.head()

### Sentiment Analysis

Outcome expected:
1. Total sentiment score
2. % of positive and negative sentiments

Like we did in our MLS exercises, I will be using VADER - rule-based sentiment analysis tool from nltk.sentiment.

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Download VADER lexicon if not already downloaded
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# this method gets the compound scores, places thresholds to classify them into positive, negative and neutral
def get_sentiment_score(text):
    sentiment_scores = sia.polarity_scores(text)
    if sentiment_scores['compound'] >=0.05 and sentiment_scores['pos'] > sentiment_scores['neg']:
        return 'positive'
    elif sentiment_scores['compound'] <= -0.05 and sentiment_scores['neg'] > sentiment_scores['pos']:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment analysis
df["sentiment"] = df["cleaned_text"].apply(get_sentiment_score)
df.head()

### Visualization
I explored the word cloud to visualize the bigger words that signify positive sentiments about the product. 

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Example: Assume df is already loaded and has columns ['text', 'sentiment']
# df = pd.read_csv("your_dataset.csv")

# Filter positive sentiment rows
positive_texts = df[df['sentiment'] == 'positive']['text']

# Join all positive texts into one string
positive_words = " ".join(positive_texts.astype(str))

# Generate the wordcloud
wordcloud = WordCloud(width=800,
                      height=400,
                      background_color="white",
                      colormap="Greens",
                      max_words=200).generate(positive_words)

# Visualize the wordcloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Positive Sentiment Words", fontsize=16)
plt.show()

## Task 4: Try a Transformer-Based Model
In this task, I tried using the transformers and datasets from HuggingFace. For the use case - Sentiment Analysis, it is more appropriate to use encoder-only model like BERT-base that understands and processes given text and outputs the analysis from it.

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

# Load dataset
df = pd.read_csv("sample_data/Product_Sentiment.csv")

# Load tokenizer and model
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create pipeline for sentiment analysis
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True,   # capture all label probabilities
    truncation=True,
    padding=True,
    device=0 if torch.cuda.is_available() else -1  # use GPU if available
)

# Run sentiment analysis
results = sentiment_pipeline(df["text"].tolist())

# Extract sentiment and scores
labels = []
scores = []

for res in results:
    # res is a list of dicts with 'label' and 'score'
    # Pick the label with max score
    best = max(res, key=lambda x: x['score'])
    labels.append(best['label'])
    scores.append({r['label']: r['score'] for r in res})

# Add results to dataframe
df["sentiment"] = labels
df["sentiment_scores"] = scores


# Save to new CSV (optional)
df.to_csv("Product_Sentiment_Analyzed.csv", index=False)
df.head()

# count the number of positive, negative and neutral sentiments
product_sentiment_count = df['sentiment'].value_counts()
print(product_sentiment_count)

## Task 5: Explore Prompt Engineering
I used ChatGPT to come up with the code that uses openai, Llama models and comapres the outcomes of few-shot and chain of thought prompts. I explored the use if LangChain pipeline to define prompt templates, llm layers and chain them for outcome. 

I could not compare outcomes at this time due to API rate limiting and quota on my personal plans.

In [ ]:
#! pip install langchain_community
! pip install --upgrade openai
import pandas as pd
from langchain.prompts import PromptTemplate
#from langchain.llms import OpenAI # Old import
from langchain_community.chat_models import ChatOpenAI # New import for chat models
# If you want to use Hugging Face (like LLaMA), import:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_core.messages import HumanMessage # Import HumanMessage

from google.colab import userdata
import os


# Set an environment variable
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Load dataset
df = pd.read_csv("sample_data/Product_Sentiment.csv")

# ---- Choose your LLM ----
# Option A: OpenAI
# llm = OpenAI(model_name="gpt-4o-mini", temperature=0) # Old way
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0) # New way for chat models

# Option B: HuggingFace (LLaMA)
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
# hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)
# llm = HuggingFacePipeline(pipeline=hf_pipeline)

# ---- Few-shot Prompt ----
few_shot_template = """
You are a sentiment analysis assistant.
Classify the following product review as Positive, Negative, or Neutral.

Examples:
Review: "I love this product! It works perfectly." → Positive
Review: "The item arrived broken and useless." → Negative
Review: "It's okay, nothing special." → Neutral

Now classify:
Review: "{review}"
Answer:
"""
few_shot_prompt = PromptTemplate(template=few_shot_template, input_variables=["review"])

# ---- Chain-of-Thought Prompt ----
cot_template = """
You are a sentiment analysis assistant.
Think step by step and explain your reasoning before giving the final classification
as Positive, Negative, or Neutral.

Review: "{review}"

First explain your reasoning.
Then give the final answer in this format:
Final Answer: <Positive/Negative/Neutral>
"""
cot_prompt = PromptTemplate(template=cot_template, input_variables=["review"])


# ---- Functions to run prompts ----
def few_shot_sentiment(text):
    # Pass a list containing a HumanMessage object to the llm
    return llm([HumanMessage(content=few_shot_prompt.format(review=text))]).content.strip()

def chain_of_thought_sentiment(text):
    # Pass a list containing a HumanMessage object to the llm
    return llm([HumanMessage(content=cot_prompt.format(review=text))]).content.strip()

# ---- Apply both approaches ----
df["few_shot_sentiment"] = df["text"].apply(few_shot_sentiment)
df["cot_sentiment"] = df["text"].apply(chain_of_thought_sentiment)

# Save results
df.to_csv("Product_Sentiment_LangChain.csv", index=False)

print(df.head())

## Observations and Insights

1. The use of basic ML models and classifiers required more mathematical approaches than the NLP and LLMs used later. 
2. Basic ML models also required us to do the training, while for the LLMs we used pre-trained models.
3. Sophisticated prompts and prompt engineering techniques should help derive a lot more information from large set of inputs.
4. Through this exercise I was able to explore and understand in detail about workings on various classifiers, models, how they train and produce outcomes and the strategies taught in the weekly classes.